In [1]:
# Required imports.
from py import calc_ubi
from py.loss_functions import loss_metrics
from py.optimize import optimize

In [2]:
AGE_CATEGORIES = ["senior", "child"]
DIS_CATEGORIES = ["dis_base", "dis_severe", "dis_enhanced"]
REGIONS = [
    "NORTH_EAST",
    "NORTH_WEST",
    "YORKSHIRE",
    "EAST_MIDLANDS",
    "WEST_MIDLANDS",
    "EAST_OF_ENGLAND",
    "LONDON",
    "SOUTH_EAST",
    "SOUTH_WEST",
    "WALES",
    "SCOTLAND",
    "NORTHERN_IRELAND",
]
categories = AGE_CATEGORIES + DIS_CATEGORIES + REGIONS

# Define bounds
AGE_BOUNDS = (40, 240)  # Child, working-age adult, senior.
DIS_BOUNDS = (0, 40)  # Base, severe, and enhanced disability UBI supplements.
GEO_BOUNDS = (-50, 50)  # Will be relative to a baseline geo.
# Skip adult which is calculated.
bounds = ([AGE_BOUNDS] * len(AGE_CATEGORIES) +
          [DIS_BOUNDS] * len(DIS_CATEGORIES) +
          [GEO_BOUNDS] * len(REGIONS))

input_dict = {category: bound for category, bound in zip(categories, bounds)}
print(input_dict)

{'senior': (40, 240), 'child': (40, 240), 'dis_base': (0, 40), 'dis_severe': (0, 40), 'dis_enhanced': (0, 40), 'NORTH_EAST': (-50, 50), 'NORTH_WEST': (-50, 50), 'YORKSHIRE': (-50, 50), 'EAST_MIDLANDS': (-50, 50), 'WEST_MIDLANDS': (-50, 50), 'EAST_OF_ENGLAND': (-50, 50), 'LONDON': (-50, 50), 'SOUTH_EAST': (-50, 50), 'SOUTH_WEST': (-50, 50), 'WALES': (-50, 50), 'SCOTLAND': (-50, 50), 'NORTHERN_IRELAND': (-50, 50)}


In [3]:
def opt(reform):
    return optimize(input_dict, "mean_pct_loss", reform,
                    verbose=False, maxiter=None, seed=0,
                    # reform_3 > reform_2 with default of 0.01, or 0.005.
                    tol=0.001
                   )

In [4]:
reform_1 = opt("reform_1")

Loss by all metrics:
 loser_share           4.515000e-01
losses                1.401599e+11
mean_pct_loss         5.190000e-02
mean_pct_loss_pwd2    5.910000e-02
gini                  2.749000e-01
dtype: float64 

Optimal mean_pct_loss: 0.0519 

Optimal solution:
 adult               124
senior              205
child                76
dis_base              0
dis_severe            0
dis_enhanced          0
NORTH_EAST            0
NORTH_WEST            0
YORKSHIRE             0
EAST_MIDLANDS         0
WEST_MIDLANDS         0
EAST_OF_ENGLAND       0
LONDON                0
SOUTH_EAST            0
SOUTH_WEST            0
WALES                 0
SCOTLAND              0
NORTHERN_IRELAND      0
dtype: int64


In [5]:
reform_2 = opt("reform_2")

Loss by all metrics:
 loser_share           4.535000e-01
losses                1.404748e+11
mean_pct_loss         5.040000e-02
mean_pct_loss_pwd2    5.580000e-02
gini                  2.727000e-01
dtype: float64 

Optimal mean_pct_loss: 0.0504 

Optimal solution:
 adult               122
senior              206
child                76
dis_base             40
dis_severe            1
dis_enhanced          0
NORTH_EAST            0
NORTH_WEST            0
YORKSHIRE             0
EAST_MIDLANDS         0
WEST_MIDLANDS         0
EAST_OF_ENGLAND       0
LONDON                0
SOUTH_EAST            0
SOUTH_WEST            0
WALES                 0
SCOTLAND              0
NORTHERN_IRELAND      0
dtype: int64


In [6]:
reform_3 = opt("reform_3")

Loss by all metrics:
 loser_share           4.526000e-01
losses                1.403028e+11
mean_pct_loss         5.040000e-02
mean_pct_loss_pwd2    5.580000e-02
gini                  2.727000e-01
dtype: float64 

Optimal mean_pct_loss: 0.0504 

Optimal solution:
 adult               117
senior              201
child                72
dis_base             40
dis_severe            4
dis_enhanced          4
NORTH_EAST            9
NORTH_WEST            8
YORKSHIRE             0
EAST_MIDLANDS         3
WEST_MIDLANDS         6
EAST_OF_ENGLAND       3
LONDON                6
SOUTH_EAST            4
SOUTH_WEST            5
WALES                 2
SCOTLAND              7
NORTHERN_IRELAND      0
dtype: int64


Check that iterations improve.

In [8]:
assert reform_2[0].fun < reform_1[0].fun
assert reform_3[0].fun < reform_2[0].fun